<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/main/15_optimization/020_Curve_Fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 최적화를 이용한 곡선적합<br>Curve fitting by Optimization


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so
np.random.seed()



데이터 생성<br>Generating data



In [ ]:
x_data = np.linspace(-10, 10)
y_true = (x_data - 1.0) * (x_data - 2.0)
noise = np.random.normal(0, 10, y_true.shape)
y_measure = y_true + noise



생성한 데이터를 표시<br>
Visualizing the data



In [ ]:
plt.plot(x_data, y_true, label="true")
plt.plot(x_data, y_measure, '.', label="with noise")
plt.grid(True)
plt.legend(loc=0)
plt.show()
plt.close();



곡선 적합 함수 선언<br>
Declare a function for curve fitting



$$
model(x, a, b, c) = a x^2 + bx + c
$$



In [ ]:
def model(x, a, b, c):
    result = x * x * a + x * b + c
    return result



In [ ]:
def calc_yhat(coefs, x):
    a, b, c = coefs
    return model(x, a, b, c)



위 함수를 시각화<br>Plot the function above



In [ ]:
def plot_curve_fitting(coefs, x):
    y_fitting = calc_yhat(coefs, x)

    plt.plot(x_data, y_fitting, label="curve fitting")
    plt.plot(x_data, y_measure, '.', label="with noise")
    plt.grid(True)
    plt.title(f"coefs = {coefs}")
    plt.legend(loc=0)
    plt.show()
    plt.close();



In [ ]:
plot_curve_fitting([1, 2, 3], x_data)



$i$'th Estimation :<br>
$i$ 번째 예측값 :



$$
{\hat{y}}_i = model(x_i,a,b,c),i=0, 1, ..., n-1
$$



## `scipy.optimize.minimize()`



Search for $(a, b, c)$ minimizing root mean square error.<br>
제곱 평균 제곱근 오차를 최소화 하는 $(a, b, c)$를 탐색.



Error of $i$'th estimation :<br>
$i$번째 예측값의 오차 :



$$
e_i=y_i - {\hat{y}}_i
$$


Calculate root mean square error<br>
오차의 제곱의 평균의 제곱근을 계산



$$
\begin{align}
    rms &=
        \sqrt {
            \frac{1}{n} \left[
                \left(y_0 - {\hat{y}}_0\right)^2 +
                \left(y_1 - {\hat{y}}_1\right)^2 +
                ...
                \left(y_{n-1} - {\hat{y}}_{n-1}\right)^2
            \right]
        } \\
    &=
        \sqrt {
            \frac{1}{n} \left[ {e_0}^2 + {e_1}^2 + ... + {e_{n-1}}^2
            \right]
        } \\
    &=
        \sqrt {
            \frac{1}{n} \left[
                \sum_{i=0}^{n-1} \left(y_i - {\hat{y}}_i\right)^2
            \right]
        } \\
    &=
        \sqrt {
            \frac{1}{n} \left(
                \sum_{i=0}^{n-1} {e_i}^2
            \right)
        }
\end{align}
$$



In [ ]:
def rms(coefs, x, y):
    y_fit = calc_yhat(coefs, x)
    error = y - y_fit
    error_square = error * error
    result = error_square.mean() ** 0.5
    return result



중간 과정을 시각화하는 비용함수를 선언<br>
Declare a cost function visualizing intermediate steps



In [ ]:
def rms_plot(coefs:np.ndarray, x:np.ndarray, y:np.ndarray) -> float:
    result = rms(coefs, x, y)

    plot_curve_fitting(coefs, x)

    return result



In [ ]:
result_minimize = so.minimize(rms_plot, [-1, 2, 30], args=(x_data, y_measure), method="Nelder-Mead")



In [ ]:
result_minimize



Result:<br>
결과:



In [ ]:
plt.plot(x_data, calc_yhat(result_minimize.x, x_data), label="curve fitting")
plt.plot(x_data, y_measure, '.', label="with noise")
plt.plot(x_data, y_true, label="true", alpha=0.5)
plt.grid(True)
plt.title(f"coefs = {result_minimize.x}")
plt.legend(loc=0)
plt.show()
plt.close();



In [ ]:
rms(result_minimize.x, x_data, y_measure)



## `scipy.optimize.curve_fit()`



A more specialized function for curve fitting<br>곡선 적합에 보다 전문화된 함수



In [ ]:
popt, pcov = so.curve_fit(model, x_data, y_measure, (1, 2, 3))
result_curve_fit = popt
result_curve_fit



Result:<br>
결과:



In [ ]:
plot_curve_fitting(result_curve_fit, x_data)



## Comparing two results<br>두 결과를 비교



rms error<br>오차 제곱 평균 제곱근



In [ ]:
rms(result_minimize.x, x_data, y_measure), rms(result_curve_fit, x_data, y_measure)



Which runs faster?<br>
더 빠른 쪽은?



In [ ]:
%%time
result_minimize = so.minimize(rms, [-1, 2, 30], args=(x_data, y_measure), method="Nelder-Mead")



In [ ]:
%%time
popt, pcov = so.curve_fit(model, x_data, y_measure, (1, 2, 3))



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

